### 1.库文件
- paddle/fluid:飞桨的主库，目前大部分的实用函数均在paddle.fluid包内。
- dygraph:动态图的类库

In [1]:
#加载飞桨、Numpy和相关类库
import paddle
import paddle.fluid as fluid
import paddle.fluid.dygraph as dygraph
from paddle.fluid.dygraph import Linear
import numpy as np
import os
import random

### 2.数据预处理
数据预处理的全部过程：主要包含五个步骤

数据导入、数据形状变换、数据集划分、数据归一化处理和封装load_data函数。

In [2]:
def load_data():
    # 从文件导入数据
    datafile = 'data/housing.data'
    data = np.fromfile(datafile, sep=' ')

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                 training_data.sum(axis=0) / training_data.shape[0]
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    global avg_values
    max_values = maximums
    min_values = minimums
    avg_values = avgs

    # 对数据进行归一化处理
    for i in range(feature_num):
        #print(maximums[i], minimums[i], avgs[i])
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

查看第一个训练样本数据

In [3]:
# 获取数据
training_data, test_data = load_data()
x = training_data[:, :-1]
y = training_data[:, -1:]


# 查看数据
print(x[0])
print(y[0])

[-0.02146321  0.03767327 -0.28552309 -0.08663366  0.01289726  0.04634817
  0.00795597 -0.00765794 -0.25172191 -0.11881188 -0.29002528  0.0519112
 -0.17590923]
[-0.00390539]


### 3.搭建神经网络

本例中模型定义的实质是定义线性回归的网络结构。飞桨建议通过创建Python类的方式完成模型网络的定义，即定义__init__函数和forward函数。

forward函数是框架指定实现前向计算逻辑的函数，程序在调用模型实例时会自动执行forward方法。在forward函数中使用的网络层需要在__init__函数中声明。实现过程分如下两步。

- 定义init函数：在类的初始化函数中声明每一层网络的实现函数。在房价预测模型中，只需要定义一层全连接层。
- 定义forward函数：构建神经网络结构，实现前向计算过程，并返回预测结果，在本任务中返回的是房价预测结果。

In [4]:
class Regressor(fluid.dygraph.Layer):
    def __init__(self, name_scope):
        super(Regressor, self).__init__(name_scope)
        name_scope = self.full_name()
        # 定义一层全连接层，输出维度是1，激活函数为None，即不使用激活函数
        self.fc = Linear(input_dim=13, output_dim=1, act=None)
    
    # 网络的前向计算函数
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

在上述代码中，name_scope变量用于调试模型时追踪多个模型的变量，在此忽略即可，飞桨1.7及之后版本不强制用户设置name_scope。

### 4.训练配置
训练配置过程包含四步：
1）指定运行训练的机器资源：以guard函数指定运行训练的机器资源，表明在with作用域下的程序均执行在本机的CPU资源上。dygraph.guard表示在with作用域下的程序会以动态图的模式执行（实时执行）。

2）声明模型实例：声明定义好的回归明模型Regressor实例，并将模型的状态设置为训练。

3）加载训练和测试数据：使用load_data函数加载训练数据和测试数据。

4）设置优化算法和学习率：优化算法采用随机梯度下降SGD，学习率设置为0.01。

In [5]:
# 定义飞桨动态图的工作环境
with fluid.dygraph.guard():
    # 声明定义好的线性回归模型
    model = Regressor("Regressor")
    # 开启模型训练模式
    model.train()
    # 加载数据
    training_data, test_data = load_data()
    # 定义优化算法，这里使用随机梯度下降-SGD
    # 学习率设置为0.01
    opt = fluid.optimizer.SGD(learning_rate=0.01, parameter_list=model.parameters())

在飞桨中，模型实例有两种状态：训练状态（.train()）和预测状态（.eval()）。训练时要执行正向计算和反向传播梯度两个过程，而预测时只需要执行正向计算。为模型指定运行状态的原因有两点：部分高级算子（例如Dropout和Batch Normalization）在两个状态执行的逻辑不同：从性能和存储空间考虑，预测状态时更节省内存，性能更好。

在上述代码可以发现，声明模型、定义优化器等操作都在with创建的fluid.dygraph.guard()上下文环境中运行，可以理解为with fluid.dygraph.guard()创建了飞桨动态图的工作环境，在该环境下完成模型声明、数据转换及模型训练等操作。

### 5.模型训练
模型训练过程采用内层循环和外层循环嵌套的方式。

内层循环负责整个数据集的一次遍历，采用分批次方式。假设数据集样本数量为1000，一个批次（batch）有10个样本，则遍历一次数据集的批次数量是100（1000/10），即内层循环需要执行100次。batch的取值会影响模型训练效果：batch过大，会增大内存消耗和计算时间，且效果并不会明显提升；batch较小，每个batch的样本数据将没有统计意义。由于房价预测模型的训练数据集较小，我们将batch设置为10.

每次内层循环都需要执行如下四个步骤。

1）数据准备：将一个批次的数据转变成np.array和内置格式。

2）前向计算：将一个批次的样本数据灌入网络中，计算输出结果。

3）计算损失函数：以前向计算结果和真实房价作为输入，通过损失函数square_error_cost计算出损失函数值（loss）

4）反向传播：执行梯度反向传播backward函数，即从后到前逐层计算每一层的梯度，并根据设置的优化算法更新参数opt.minimize

外层循环定义遍历数据集的次数，通过参数EPOCH_NUM设置。具体定义训练过程如下：

In [6]:
with dygraph.guard():
    EPOCH_NUM = 10   # 设置外层循环次数
    BATCH_SIZE = 10  # 设置batch大小
    
    # 定义外层循环
    for epoch_id in range(EPOCH_NUM):
        # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
        np.random.shuffle(training_data)
        # 将训练数据进行拆分，每个batch包含10条数据
        mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
        # 定义内层循环
        for iter_id, mini_batch in enumerate(mini_batches):
            # 获得当前批次训练数据
            x = np.array(mini_batch[:, :-1]).astype('float32')
            # 获得当前批次训练标签（真实房价）
            y = np.array(mini_batch[:, -1:]).astype('float32')
            # 将numpy数据转为飞桨动态图variable形式
            house_features = dygraph.to_variable(x)
            prices = dygraph.to_variable(y)
            
            # 前向计算
            predicts = model(house_features)
            
            # 计算损失
            loss = fluid.layers.square_error_cost(predicts, label=prices)
            avg_loss = fluid.layers.mean(loss)
            if iter_id%20==0:
                print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
            
            # 反向传播
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.minimize(avg_loss)
            # 清除梯度
            model.clear_gradients()
    # 保存模型
    fluid.save_dygraph(model.state_dict(), 'LR_model')

epoch: 0, iter: 0, loss is: [0.09032011]
epoch: 0, iter: 20, loss is: [0.14171907]
epoch: 0, iter: 40, loss is: [0.00680798]
epoch: 1, iter: 0, loss is: [0.05885943]
epoch: 1, iter: 20, loss is: [0.18440022]
epoch: 1, iter: 40, loss is: [0.05989945]
epoch: 2, iter: 0, loss is: [0.06258215]
epoch: 2, iter: 20, loss is: [0.03788795]
epoch: 2, iter: 40, loss is: [0.12838918]
epoch: 3, iter: 0, loss is: [0.07172807]
epoch: 3, iter: 20, loss is: [0.0517186]
epoch: 3, iter: 40, loss is: [0.02951107]
epoch: 4, iter: 0, loss is: [0.0415845]
epoch: 4, iter: 20, loss is: [0.06107662]
epoch: 4, iter: 40, loss is: [0.07843176]
epoch: 5, iter: 0, loss is: [0.06174529]
epoch: 5, iter: 20, loss is: [0.05816273]
epoch: 5, iter: 40, loss is: [0.02456109]
epoch: 6, iter: 0, loss is: [0.02674029]
epoch: 6, iter: 20, loss is: [0.0410394]
epoch: 6, iter: 40, loss is: [0.02404088]
epoch: 7, iter: 0, loss is: [0.11601124]
epoch: 7, iter: 20, loss is: [0.06512068]
epoch: 7, iter: 40, loss is: [0.04860499]
epo

通过代码运行结果可以发现，损失值呈总体下降趋势。

### 6.保存并测试模型
首先，我们将模型当前的参数数据model.state_dict()保存到文件中（通过参数指定保存的文件名LR_model），以备预测或校验的程序调用，代码如下：

In [7]:
# 定义飞桨动态图工作环境
with fluid.dygraph.guard():
    # 保存模型参数，文件名为LR_model
    fluid.save_dygraph(model.state_dict(), 'LR_model')
    print("模型保存成功，模型参数保存在LR_model中")
    

模型保存成功，模型参数保存在LR_model中


然后就可以对模型进行测试了，测试过程与在应用场景中使用模型的过程一直，主要可分成如下三个步骤：

1）配置模型预测是机器资源。

2）将训练好的模型参数加载到模型实例中。由两个语句完成，第一句是从文件中获取模型参数；第二句是将参数内容加载到模型。加载完毕后，需要将模型的状态调整为校验（evaluation）。上文中提到，训练状态的模型需要同时支持正向计算和反向传导梯度，模型的实现较为臃肿，而校验和预测的模型只需要支持正向计算，模型的实现更加简单，性能更好。

3）将待预测的样本特征输入模型中，打印输出的预测结果。

通过load_one_example函数从数据集中抽取一条样本作为测试样本，具体实现代码如下：

In [8]:
def load_one_example(data_dir):
    f = open(data_dir, 'r')
    datas = f.readlines()
    # 选择倒数第10条数据用于测试
    tmp = datas[-10]
    tmp = tmp.strip().split()
    one_data = [float(v) for v in tmp]

    # 对数据进行归一化处理
    for i in range(len(one_data)-1):
        one_data[i] = (one_data[i] - avg_values[i]) / (max_values[i] - min_values[i])

    data = np.reshape(np.array(one_data[:-1]), [1, -1]).astype(np.float32)
    label = one_data[-1]
    return data, label

然后开始测试

In [9]:
with dygraph.guard():
    # 参数为保存模型参数的文件地址
    model_dict, _ = fluid.load_dygraph('LR_model')
    model.load_dict(model_dict)
    model.eval()

    # 参数为数据集的文件地址
    test_data, label = load_one_example('./data/housing.data')
    # 将数据转为动态图的variable格式
    test_data = dygraph.to_variable(test_data)
    results = model(test_data)

    # 对结果做反归一化处理
    results = results * (max_values[-1] - min_values[-1]) + avg_values[-1]
    print("Inference result is {}, the corresponding label is {}".format(results.numpy(), label))

Inference result is [[14.460349]], the corresponding label is 19.7


运行上述代码，我们可以比较“模型预测值”和“真实房价”：模型的预测效果与真实房价接近。